# Ideas for the Project

Only Scrape from "Data Scientist" and "Data Analyst" DO NOT include Business Analyst or prefixes like Senior/Junior

## Warning
These functions may be buggy and need fixing but I hope this jumpstarts the data collection process for you. Feel free to edit the functions if they aren't doing what you need properly

In [243]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import nltk
import numpy as np
import re
from nltk.corpus import wordnet
from selenium import webdriver

<Response [200]>

In [181]:
def raw_text(url):
    """Takes a URL as input and performs web scrapping to retrieve the body of the
    webpage (in this case a Linkedin ad)"""
    ad = requests.get(url) #Retrieve webpage
    Html = BeautifulSoup(ad.text, 'html.parser') #Convert html into a nicer format
    text_body = Html.find_all('div', 
                              {'class':"show-more-less-html__markup show-more-less-html__markup--clamp-after-5"})
    text_body = text_body[0].text
    return text_body
def clean_text(doc):
    """Take an unstructured document and tokenize it into a list of words. 
    Then standardize it by lowercasing and lemmatizing each word"""
    words = re.findall(r'(?:[a-zA-Z]|#|"+")+',doc) #Find all alphabetical words (Preserve + and # for C++ and C#)
    stopwords = nltk.corpus.stopwords.words("english")
    words = [i.lower() for i in words] #Lowercase all words
    words = [i for i in words if i not in stopwords] #Filter out stopwords
    tag_words = nltk.pos_tag(words) #Begin lemmatizing by tagging each word
    tag_words = [(i, wordnet_pos(j)) for (i, j) in tag_words] #Convert the tags into something the lemmatizer understands
    lemmatizer = nltk.WordNetLemmatizer()
    clean_words = [lemmatizer.lemmatize(i, j) for i, j in tag_words] #Lemmatize the words
    #Document should be cleaned up
    return clean_words
def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN) #Function created by Bo Ning in Week 6-2
def lang_count(text):
    """ Take a body of clean text and count the number of programming languages present"""
    languages = ['python','r','sql','sa','c',
                 'c++','c#','java','javascript',
                 'julia','matlab','swift','tableau'
                'microsoft','github'] #SAS turns into sa after lemmatization
    #ADD MORE LANGUAGES IF NECESSARY
    count = sum([i in text for i in languages]) #Check if each language is in the ad
    #And sum the number of programming languages present
    return count

In [276]:
def get_salary(text):
    """From a body of raw text, retrieve the salary"""
    salaries = re.findall(r"(\d+\,\d+\.\d{1,2})",text) 
    if salaries != []:
        return salaries[-1] #Let's work with the maximum salary
    else:
        salaries = re.findall(r"(\d+\,\d+)",text) 
        if salaries != []:
            return salaries[-1]
        else:
            return "NaN"
def ML_skill(text):
    """Using a body of clean text, check whether the words machine learning is present
    to see if it is a required skill"""
    return str(int('machine' in text and 'learning' in text))
def get_edu(text):
    """Using a body of raw text, retrieve the education level"""
    if "Master" in text and "Bachelor" in text:
        return "4" #Category where Bachlor's is minimum but higher level preferred
    elif "PhD" in text:
        return "3"
    elif "Master" in text:
        return "2"
    elif "Bachelor" in text:
        return "1"
    else:
        return "0" #No education specified
def ad_type(text):
    """Using a body of clean text, check whether this is an ad for data analyst or scientists"""
    return str(int("science" in text))
def benefits(text):
    """Using a body of raw text, check if benefits are included"""
    return str(int('Benefits' in text or 'benefits' in text))
def exp(text):
    """Using a body of raw text, check if experience is required/preferred"""
    return str(int("experience" in text))

In [250]:
def collect_data(url):
    """Input a URL for a Linkedin Ad and retrieve all relevant data"""
    raw = raw_text(url)
    clean = clean_text(raw)
    return {'Languages':lang_count(clean),
            'Salary':get_salary(raw),
            'Machine Learning':ML_skill(clean),
            'Education':get_edu(raw),
            'Type': ad_type(clean),
            'Benefits':benefits(raw),
            'Experience':exp(raw)}

In [254]:
raw = raw_text('https://www.linkedin.com/jobs/view/3492106426/?alternateChannel=search&refId=rKjt309G%2Bcgm96lG7K9V4w%3D%3D&trackingId=jRvZLlqPS%2FSxMQpDc9liGQ%3D%3D')

In [262]:
words = re.findall(r'(?:[a-zA-Z]|#|"+")+',raw)

In [277]:
collect_data('https://www.linkedin.com/jobs/view/3492106426/?alternateChannel=search&refId=rKjt309G%2Bcgm96lG7K9V4w%3D%3D&trackingId=jRvZLlqPS%2FSxMQpDc9liGQ%3D%3D')

{'Languages': 2,
 'Salary': '45,000',
 'Machine Learning': '0',
 'Education': '1',
 'Type': '1',
 'Benefits': '1',
 'Experience': '1'}

In [274]:
re.findall(r"(\d+\,\d+\.\d{0,1,2})",raw) 

[]

In [273]:
raw

"\nOverviewAs a leading global media network, Omnicom Media Group (OMG) sets itself apart with an agile, client-first approach that helps businesses thrive today and into the future. With more than 21,000 employees globally, OMG has the talent, expertise and clout to deliver unprecedented levels of innovation for our clients. The group is comprised of some of the world's most awarded agencies. From leadership to entry-level talent, we are a diverse mix of ages, genders, ethnicities, skill sets and backgrounds. This role will be on a highly visible team, fully dedicated to the success of our client, L’Oréal.For more than a century, L’Oréal has devoted its energy, innovation, and scientific excellence solely to one business: Beauty. Their goal is to offer each and every person around the world the best of beauty in terms of quality, efficacy, safety, sincerity and responsibility to satisfy all beauty needs and desires in their infinite diversity.Qualifications:Must have hands-on expertis

In [275]:
re.findall(r"(\d+\,\d+)",raw) 

['21,000', '30,000', '45,000']